<a href="https://colab.research.google.com/github/bskang8/CVPR2023_Project/blob/main/evi_gov_notice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install langchain
!pip install sentence-transformers
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.1/448.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/

In [58]:
!pip install xlrd
!pip install openpyxl
!pip install pandas

In [59]:
import os
import pandas as pd

from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [9]:
file_path = '/content/drive/MyDrive/test_folder/dataset/환경부 공지'
file_names = os.listdir(file_path)
chunk_documents = []

cut_len = 385
idx = 1
for file in file_names:
    whole_path = os.path.join(file_path, file)
    with open(whole_path, 'r', encoding='UTF8') as f:
        text = f.read()

    time = 1
    while time*cut_len <= len(text):
        b_time = time - 1
        chunk_text = text[b_time*cut_len:time*cut_len]
        chunk_documents.append(Document(page_content=chunk_text, metadata={'file_name':file, 'file_num': idx}))
        time += 1
    if (time-1)*cut_len <= len(text):
        chunk_text = text[(time-1)*cut_len:len(text)]
        chunk_documents.append(Document(page_content=chunk_text, metadata={'file_name':file, 'file_num': idx}))

In [14]:
embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large')

In [19]:
vectordb = Chroma.from_documents(documents=chunk_documents, embedding=embeddings)

In [54]:
doc_retriever = vectordb.as_retriever(search_kwargs={"k":5})

In [75]:
def const_ref_doc(ref_docs):
  cont_ref = ''
  idx = 1
  for doc in ref_docs:
    cont_ref += "===== "
    cont_ref += ' Reference: ' + str(idx)
    cont_ref += " =====\n"
    cont_ref += doc.page_content
    idx += 1
    cont_ref += '\n'
    cont_ref += 'Meta Data : ' + doc.metadata['file_name']
    cont_ref += '\n'
  return cont_ref

In [63]:
file_name = '/content/drive/MyDrive/test_folder/dataset/qna/환경부 질의응답.xlsx'
df = pd.read_excel(file_name)
qs_list = df['질문'].tolist()

In [87]:
s_path = '/content/drive/MyDrive/test_folder/dataset/contents'
os.makedirs(s_path, exist_ok=True)

for qs in qs_list:
  ref_docs = doc_retriever.get_relevant_documents(qs)
  ref = const_ref_doc(ref_docs)

  content = '#### Instruction: \n'
  content += '아래는 질문과 Reference 내용이다. Reference 내용을 기반으로 질문의 대답을 작성해 주시오.\n\n'
  content += '#### 질문: \n'
  content += qs
  content += '\n'
  content += '#### Reference: \n'
  content += ref
  content += '\n\n'
  content += '#### 대답: \n'

  qs_sep = qs.split('/')
  if len(qs_sep) > 1:
    qs_re = ''
    for s_q in qs_sep:
      qs_re += s_q
      qs_re += '_'
    qs_re[:len(qs_re)-1]
  else:
    qs_re = qs_sep[0]

  s_file_path = os.path.join(s_path, qs_re+'.txt')
  with open(s_file_path, 'w', encoding='UTF8') as f:
    f.write(content)